In [ ]:
# этот скрипт делает графики интервалов 1D, 1H, 5m, 1m, 5s с индикаторами, которые чат попросил для анализа.

In [ ]:
#%pip install numpy
#%pip install pandas
#%pip install mplfinance
#%pip install matplotlib
#%pip install ta
#%pip install pandas_ta

In [ ]:
import numpy as np
import pandas as pd
import mplfinance as mpf
import matplotlib.pyplot as plt
#import ta # https://github.com/bukosabino/ta
import pandas_ta as ta

In [ ]:
# параметры:
symbol = 'BTCUSD'
charts_at = '2024-05-29 08:10' # время окончания графика

In [ ]:
# в api chatgpt можно передавать картинки размером до 2000 на 768 точек.
# (src: https://platform.openai.com/docs/guides/vision/managing-images : "For high res mode, the short side of the image 
# should be less than 768px and the long side should be less than 2,000px.")
# при этом только картинки 512 на 512 обрабатываются одним блоком. картинки большего размера разбиваются на квадраты
# по 512 на 512 точек.
# в связи с этим я вижу несколько вариантов для размера картинок, которые имеют смысл:
# 1) 512 на 512 - самый дешевый варинт. но сложно вместить всю информацию, которую хочется передать.
# 2) 1024 на 512 - увеличивает стоимость в 2 раза. позволяет большую историю передать.
# 3) 512 на 768 - увеличивает стоимость в 2 раза. позволяет передать больше индикаторов, которые добавляются снизу.
# 4) 1024 на 768 - увеличивает стоимость в 4 раза. среднее количество истории и есть место для индикаторов.
# 5) 1536 на 512 - увеличивает стоимость в 3 раза. достаточно истории, но мало места для индикаторов.
# 6) 1536 на 768 - увеличивает стоимость в 6 раз. достаточно истории и максимум места для индикаторов.
# 7) 2000 на 768 - увеличивает стоимость в 8 раз. максимальный вариант.
# я не уверен, что есть смысл в варианте 7, т.к. на мой вкус такой график выглядит не лучше чем вариант 6. 
# в идеале надо поэкспериментировать.

# DPI в matplotlib = 96 пикселов на дюйм. а размер графика задается в дюймах. поэтому делаю пересчет.
dpi = 96
# 1)
# width_in_inches = 512 / dpi
# height_in_inches = 512 / dpi

# 2)
#width_in_inches = 1024 / dpi
#height_in_inches = 512 / dpi

# 3)
#width_in_inches = 512 / dpi
#height_in_inches = 768 / dpi

# 4)
#width_in_inches = 1024 / dpi
#height_in_inches = 768 / dpi

# 5)
#width_in_inches = 1536 / dpi
#height_in_inches = 512 / dpi

# 6)
width_in_inches = 1536 / dpi
height_in_inches = 768 / dpi

# 7)
#width_in_inches = 2000 / dpi
#height_in_inches = 768 / dpi

# количество свечей на графике
points = 200

### цены за gpt-4o:

- стоимость ввода: $0.005 за 1к

- стоимость вывода: $0.015 за 1к

- стоимость картинки: $0.001275 за 512х512

### оценка стоимости запроса:

ввод: 100 токенов = 0.1 * $0.005 = $0.0005

вывод: 300 токенов = 0.3 * $0.015 = $0.0045

картинка: от $0.001275 до $0.005525


### вывод:

добавление картинки к запросу увеличивает стоимость на от 25% до 110%


note: batch 50% discount: https://platform.openai.com/docs/api-reference/batch

In [ ]:
# date_time - string,
def get_df(date_time, period, length = 1000):
    """
    Загрузить датафрейм с диска и взять с него слайс.
    
    Параметры:
    date_time : string
        Например, 2024-05-29 08:10. Время не обязательно должно попадать в точку графика (будет взята ближайшая точка).
    period : string
        Допустимые значения: '5S' - 5 секунд, '1T' - 1 minute, '5T' - 5 minutes, '1H' - 1 hour, 'D' - 1 day.
    length : number
        Сколько точек надо в датафрейме.
    """
    if period == '5S': # пятисекундные свечи
        # '2024-05-29 08:10' -> '2024-05-29'
        date = date_time[:10]
        filename = f'npz/{symbol}/{symbol}{date}.npz'
    else:
        filename = f'npz/{symbol}_{period}.npz'

    # зачитывание в формате numpy
    data = np.load(filename, allow_pickle=True)['data']
    # преобразование в формат pandas
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

    # Преобразование 'timestamp' в datetime и установка его как индекс DataFrame
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)

    # Преобразование всех столбцов с ценами и объемом в числовой формат
    for column in ['open', 'high', 'low', 'close', 'volume']:
        df[column] = pd.to_numeric(df[column], errors='coerce')

    # поиск интересной точки.
    date_to_find = date_to_find = pd.Timestamp(date_time)
    index_position = df.index.get_indexer([date_to_find], method='nearest')[0]
    # беру часть точек, чтобы не напрягать либу технического анализа большим количеством точек.
    df = df[index_position-length:index_position]
    return df

In [ ]:
df = get_df(charts_at, '5S')

In [ ]:
# # вывод графика просто, чтобы посмотреть что считалось с диска.
# df[['open', 'high', 'low', 'close']][-points:].plot(figsize=(12,5))
# # вывод датафрейма, чтобы проверить что загрузилось.
# df

In [ ]:
# # стиль графика для mpl. пытаюсь сделать что-то похожее на светлую тему tradingview.
# customstyle = mpf.make_mpf_style(base_mpf_style='yahoo', facecolor='w')

# # wconfig может быть полезн, чтобы считать текущие значения width_config в mpf, чтобы понять что менять.
# #wconfig = {}

# # https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html#ta.volatility.DonchianChannel
# dc = ta.donchian(df[-points-20:]['high'], df[-points-20:]['low'])

# # https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html#ta.momentum.StochRSIIndicator
# rsi = ta.rsi(df[-points-20:]['close'])

# # https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html#ta.ema
# ema20 = ta.ema(df[-points-20:]['close'], 20)
# ema50 = ta.ema(df[-points-50:]['close'], 50)
# ema100 = ta.ema(df[-points-100:]['close'], 100)
# ema200 = ta.ema(df[-points-200:]['close'], 200)

# # https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html#ta.trend.MACD
# #macd = ta.trend.MACD(df[-points-26:]['close'], 26)
# macddf = ta.macd(df[-points-200:]['close'], fast=12, slow=26, signal=9, min_periods=None, append=True)

# # Bollinger Bands
# bb = ta.bbands(df[-points-200:]['close'], length=20, std=2)
# bblow, bbmid, bbup = bb[bb.columns[0]], bb[bb.columns[1]], bb[bb.columns[2]]

# # добавление графиков к mpf https://github.com/matplotlib/mplfinance/blob/master/examples/addplot.ipynb
# apdict = [
#     #mpf.make_addplot(dc['DCL_20_20'][-points:], color='gray'),
#     #mpf.make_addplot(dc['DCU_20_20'][-points:], color='gray'),

#     # Bollinger Bands
#     mpf.make_addplot(bbup[-points:], color='red'),
#     mpf.make_addplot(bbmid[-points:], color='blue'),
#     mpf.make_addplot(bblow[-points:], color='green'),
    
#     #mpf.make_addplot(ema20[-points:], color='yellow'),
#     #mpf.make_addplot(ema50[-points:], color='orange'),
#     #mpf.make_addplot(ema100[-points:], color='pink'),
#     #mpf.make_addplot(ema200[-points:], color='red'),
    
#     mpf.make_addplot(macddf[macddf.columns[0]][-points:], panel=2, secondary_y=False),
#     mpf.make_addplot(macddf[macddf.columns[2]][-points:], panel=2, linestyle='dotted', ylabel='MACD'),
#     mpf.make_addplot(macddf[macddf.columns[1]][-points:], panel=2, secondary_y=False, type='bar'),
    
#     mpf.make_addplot(rsi[-points:], panel=1, secondary_y=True),
#     mpf.make_addplot(ta.ema(rsi[-points-20:], 20)[-points:], panel=1, secondary_y=True),
#     ]

# # отрисовка.
# fix, ax = mpf.plot(
#     df[-points:],
#     type='candle',
#     volume=True, 
#     style=customstyle,
#     figsize=(width_in_inches, height_in_inches),
#     returnfig=True, # это надо, чтобы mpf.plot вернул fix, ax.
#     # разворот подписей по оси X, т.к. в документации openai написано, что повернутый текст он плохо читает (наверное 
#     # умышленно порезана модель, чтобы нельзя было ее использовать для распознавания капчи)
#     xrotation=0, 
#     #return_width_config=wconfig,
#     update_width_config={'volume_width': 0.75, 'candle_width': 0.75, 'candle_linewidth': 1},
#     addplot=apdict
#     )

# ax[0].margins(x=0) # убрать отступы слева и справа на графике.

# # отрисовка MACD в отдельном графике. 
# # (взято из примеров https://github.com/twopirllc/pandas-ta/blob/main/examples/example.ipynb )
# #macddf[[macddf.columns[0], macddf.columns[2]]].plot(figsize=(16, 2), linewidth=1.3)
# #macddf[macddf.columns[1]].plot.area(stacked=False, color=["silver"], linewidth=1, grid=True).axhline(y=0, color="black", lw=1.1)

# # вывод width_config.
# #wconfig

In [ ]:
#help(ta.bbands)

In [ ]:
# стиль графика для mpl. пытаюсь сделать что-то похожее на светлую тему tradingview.
customstyle = mpf.make_mpf_style(base_mpf_style='yahoo', facecolor='w')

In [ ]:
rsi = ta.rsi(df[-points-7:]['close'], 7)
macddf = ta.macd(df[-points-200:]['close'], fast=6, slow=13, signal=5, min_periods=None, append=True)

# Bollinger Bands
bb = ta.bbands(df[-points-200:]['close'], length=20, std=2)
bblow, bbmid, bbup = bb[bb.columns[0]], bb[bb.columns[1]], bb[bb.columns[2]]

# добавление графиков к mpf https://github.com/matplotlib/mplfinance/blob/master/examples/addplot.ipynb
apdict = [
    # Bollinger Bands
    mpf.make_addplot(bbup[-points:], color='red'),
    mpf.make_addplot(bbmid[-points:], color='blue'),
    mpf.make_addplot(bblow[-points:], color='green'),
    
    mpf.make_addplot(macddf[macddf.columns[0]][-points:], panel=2, secondary_y=False),
    mpf.make_addplot(macddf[macddf.columns[2]][-points:], panel=2, linestyle='dotted', ylabel='MACD'),
    mpf.make_addplot(macddf[macddf.columns[1]][-points:], panel=2, secondary_y=False, type='bar'),
    
    mpf.make_addplot(rsi[-points:], panel=1, secondary_y=True),
    mpf.make_addplot(ta.ema(rsi[-points-20:], 20)[-points:], panel=1, secondary_y=True),
    ]

# отрисовка.
fix, ax = mpf.plot(
    df[-points:],
    type='candle',
    volume=True, 
    style=customstyle,
    figsize=(width_in_inches, height_in_inches),
    returnfig=True, # это надо, чтобы mpf.plot вернул fix, ax.
    # разворот подписей по оси X, т.к. в документации openai написано, что повернутый текст он плохо читает (наверное 
    # умышленно порезана модель, чтобы нельзя было ее использовать для распознавания капчи)
    xrotation=0, 
    #return_width_config=wconfig,
    update_width_config={'volume_width': 0.75, 'candle_width': 0.75, 'candle_linewidth': 1},
    addplot=apdict,
    savefig=dict(fname='img/5S.png', dpi=96, bbox_inches='tight')
    )

ax[0].margins(x=0) # убрать отступы слева и справа на графике.

In [ ]:
df = get_df(charts_at, '1T')

In [ ]:
# # вывод графика просто, чтобы посмотреть что считалось с диска.
# df[['open', 'high', 'low', 'close']][-points:].plot(figsize=(12,5))
# # вывод датафрейма, чтобы проверить что загрузилось.
# df

In [ ]:
#help(ta.vwap)

In [ ]:
atr = ta.atr(df[-points-7:]['high'],df[-points-7:]['low'],df[-points-7:]['close'], 7)
ema21 = ta.ema(df[-points-21:]['close'], 21)
vwap = ta.vwap(df[-points:]['high'],df[-points:]['low'],df[-points:]['close'],df[-points:]['volume'])

# добавление графиков к mpf https://github.com/matplotlib/mplfinance/blob/master/examples/addplot.ipynb
apdict = [
    mpf.make_addplot(atr[-points:], panel=2, color='red'),
    mpf.make_addplot(ema21[-points:], color='black'),
    mpf.make_addplot(vwap[-points:], color='blue'),
    ]

# отрисовка.
fix, ax = mpf.plot(
    df[-points:],
    type='candle',
    volume=True, 
    style=customstyle,
    figsize=(width_in_inches, height_in_inches),
    returnfig=True, # это надо, чтобы mpf.plot вернул fix, ax.
    # разворот подписей по оси X, т.к. в документации openai написано, что повернутый текст он плохо читает (наверное 
    # умышленно порезана модель, чтобы нельзя было ее использовать для распознавания капчи)
    xrotation=0, 
    #return_width_config=wconfig,
    update_width_config={'volume_width': 0.75, 'candle_width': 0.75, 'candle_linewidth': 1},
    addplot=apdict,
    savefig=dict(fname='img/1T.png', dpi=96, bbox_inches='tight')
    )

ax[0].margins(x=0) # убрать отступы слева и справа на графике.

In [ ]:
df = get_df(charts_at, '5T')

In [ ]:
#help(ta.ichimoku)

In [ ]:
cci = ta.cci(df[-points-14:]['high'],df[-points-14:]['low'],df[-points-14:]['close'], 14)
obv = ta.obv(df[-points:]['close'],df[-points-20:]['volume'])
(ichimoku, _) = ta.ichimoku(df[-2*points:]['high'],df[-2*points:]['low'],df[-2*points:]['close'])
# how to fill: https://medium.com/@corinneroosen/create-an-ichimoku-cloud-with-mplfinance-e9cb5dc28a71
fill_up = dict(y1 = ichimoku['ISA_9'][-points:].values, y2 = ichimoku['ISB_26'][-points:].values, where = ichimoku['ISA_9'][-points:] >= ichimoku['ISB_26'][-points:], alpha = 0.5, color = 'honeydew')
fill_down = dict(y1 = ichimoku['ISA_9'][-points:].values, y2 = ichimoku['ISB_26'][-points:].values, where = ichimoku['ISA_9'][-points:] < ichimoku['ISB_26'][-points:], alpha = 0.5, color = 'mistyrose')

# добавление графиков к mpf https://github.com/matplotlib/mplfinance/blob/master/examples/addplot.ipynb
apdict = [
    mpf.make_addplot(cci[-points:], panel=1, color='blue'),
    mpf.make_addplot(obv[-points:], color='black'),

    mpf.make_addplot(ichimoku['ITS_9'][-points:], color = 'blue', width = 1),
    mpf.make_addplot(ichimoku['IKS_26'][-points:], color = 'red', width = 1),
    mpf.make_addplot(ichimoku['ISA_9'][-points:], color = 'lightgreen', width = 0.5),
    mpf.make_addplot(ichimoku['ISB_26'][-points:], color = 'lightcoral', width = 0.5),
    mpf.make_addplot(ichimoku['ICS_26'][-points:], color = 'green', width = 1)
    ]

# отрисовка.
fix, ax = mpf.plot(
    df[-points:],
    type='candle',
    volume=True, 
    style=customstyle,
    figsize=(width_in_inches, height_in_inches),
    returnfig=True, # это надо, чтобы mpf.plot вернул fix, ax.
    # разворот подписей по оси X, т.к. в документации openai написано, что повернутый текст он плохо читает (наверное 
    # умышленно порезана модель, чтобы нельзя было ее использовать для распознавания капчи)
    xrotation=0, 
    #return_width_config=wconfig,
    update_width_config={'volume_width': 0.75, 'candle_width': 0.75, 'candle_linewidth': 1},
    addplot=apdict,
    fill_between = [fill_up, fill_down],
    savefig=dict(fname='img/5T.png', dpi=96, bbox_inches='tight')
    )

ax[0].margins(x=0) # убрать отступы слева и справа на графике.

In [ ]:
df = get_df(charts_at, '1H')

In [ ]:
#help(ta.adx)

In [ ]:
rsi = ta.rsi(df[-points-20:]['close'])
macddf = ta.macd(df[-points-200:]['close'], fast=12, slow=26, signal=9, min_periods=None, append=True)
adx = ta.adx(df[-points-14:]['high'],df[-points-14:]['low'],df[-points-14:]['close'], 14)
# Bollinger Bands
bb = ta.bbands(df[-points-20:]['close'], length=20, std=2)
bblow, bbmid, bbup = bb[bb.columns[0]], bb[bb.columns[1]], bb[bb.columns[2]]

# добавление графиков к mpf https://github.com/matplotlib/mplfinance/blob/master/examples/addplot.ipynb
apdict = [
    # Bollinger Bands
    mpf.make_addplot(bbup[-points:], color='red'),
    mpf.make_addplot(bbmid[-points:], color='lightblue'),
    mpf.make_addplot(bblow[-points:], color='green'),

    mpf.make_addplot(macddf[macddf.columns[0]][-points:], panel=2, secondary_y=False),
    mpf.make_addplot(macddf[macddf.columns[2]][-points:], panel=2, linestyle='dotted', ylabel='MACD'),
    mpf.make_addplot(macddf[macddf.columns[1]][-points:], panel=2, secondary_y=False, type='bar'),
    
    mpf.make_addplot(rsi[-points:], panel=1, secondary_y=True),
    mpf.make_addplot(ta.ema(rsi[-points-20:], 20)[-points:], panel=1, secondary_y=True),
    
    mpf.make_addplot(adx[adx.columns[0]][-points:], panel=0, color='blue'),
    ]

# отрисовка.
fix, ax = mpf.plot(
    df[-points:],
    type='candle',
    volume=True, 
    style=customstyle,
    figsize=(width_in_inches, height_in_inches),
    returnfig=True, # это надо, чтобы mpf.plot вернул fix, ax.
    # разворот подписей по оси X, т.к. в документации openai написано, что повернутый текст он плохо читает (наверное 
    # умышленно порезана модель, чтобы нельзя было ее использовать для распознавания капчи)
    xrotation=0, 
    #return_width_config=wconfig,
    update_width_config={'volume_width': 0.75, 'candle_width': 0.75, 'candle_linewidth': 1},
    addplot=apdict,
    savefig=dict(fname='img/1H.png', dpi=96, bbox_inches='tight')
    )

ax[0].margins(x=0) # убрать отступы слева и справа на графике.